### Exporting Logs in NER training

In [ ]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.training import CoNLL
from pyspark.sql import SparkSession

To use S3 to store training logs, we have two options:
- Defining S3 path information as well as AWS credentials while starting spark
- Defining S3 path information in runtime and AWS credentials while starting spark (Available since spark-nlp 4.1.0)

In [ ]:
spark = SparkSession.builder \
    .appName("SparkNLP") \
    .master("local[*]") \
    .config("spark.driver.memory", "12G") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryoserializer.buffer.max", "2000M") \
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.jars", "sparknlp.jar") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.1.0") \
    .config("spark.jsl.settings.aws.credentials.access_key_id", "MY_ACCESS_KEY_ID") \
    .config("spark.jsl.settings.aws.credentials.secret_access_key", "MY_SECRET_ACCESS_KEY") \
    .config("spark:spark.jsl.settings.aws.credentials.session_token", "MY_SESSION_TOKEN") \
    .config("spark.jsl.settings.aws.region", "MY_AWS_REGION") \
    .getOrCreate()

print("Apache Spark version: {}".format(spark.version))

Apache Spark version: 3.2.0


Please check how to start a spark session with spark-nlp based on your environment [here]( https://github.com/JohnSnowLabs/spark-nlp#usage)

### Training NER DL

In [ ]:
training_data = CoNLL().readDataset(spark, './test_ner_dataset.txt')
training_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|John Smith works ...|[{document, 0, 35...|[{document, 0, 35...|[{token, 0, 3, Jo...|[{pos, 0, 3, NNP,...|[{named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+



In [ ]:
embeddings = WordEmbeddingsModel.pretrained("glove_100d")
ready_data = embeddings.transform(training_data).cache()

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


The example below defines an S3 in runtime:

In [ ]:
ner_tagger = NerDLApproach() \
                .setInputCols("sentence", "token", "embeddings") \
                .setLabelColumn("label") \
                .setOutputCol("ner") \
                .setMaxEpochs(1) \
                .setMaxEpochs(5) \
                .setRandomSeed(0) \
                .setVerbose(2) \
                .setDropout(0.8) \
                .setBatchSize(18) \
                .setEnableOutputLogs(True)
                .setOutputLogsPath("s3://my_bucket/my_path/ner_logs")

In [ ]:
ner_tagger.fit(ready_data)

NerDLModel_4cc29d1aa9e3

Before spark-nlp 4.1.0, in addition to AWS credentials, we needed to define the configuration below for spark session:

In [ ]:
spark.conf.set("spark.jsl.settings.aws.s3_bucket", "MY_S3_BUCKET")
spark.conf.set("spark.jsl.settings.annotator.log_folder", "s3://my_path/ner_logs") #yes, without my_bucket

This configuration is still available in 4.1.0, but the path defined in `setOutputLogsPath` takes precedence.